<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Personalized_Itinerary_Generator_Based_Radius/ModelTraining1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ace-tools

In [ ]:
!pip install geopy

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random

In [ ]:
# Load the datasets
hotels_df = pd.read_csv("colomboHotels_processed.csv").dropna(subset=["latitude", "longitude"])
attractions_df = pd.read_csv("colombo_attractions_processed.csv").dropna(subset=["latitude", "longitude"])
restaurants_df = pd.read_csv("colomboRestaurants_processed.csv").dropna(subset=["latitude", "longitude"])

# Function to calculate distance between two coordinates
def haversine_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Function to suggest top hotels in a place
def suggest_hotels():
    return hotels_df.sort_values(by=["rating", "priceRange_LKR"], ascending=[False, True]).head(5)

# Function to get nearby attractions within range
def get_nearby_attractions(hotel_lat, hotel_lon, max_range_km):
    attractions_df["distance"] = attractions_df.apply(
        lambda row: haversine_distance((hotel_lat, hotel_lon), (row["latitude"], row["longitude"])), axis=1
    )
    return attractions_df[attractions_df["distance"] <= max_range_km].sort_values(by="rating", ascending=False)

# Function to get nearby restaurants within range
def get_nearby_restaurants(hotel_lat, hotel_lon, max_range_km):
    restaurants_df["distance"] = restaurants_df.apply(
        lambda row: haversine_distance((hotel_lat, hotel_lon), (row["latitude"], row["longitude"])), axis=1
    )
    return restaurants_df[restaurants_df["distance"] <= max_range_km].sort_values(by="rating", ascending=False)

# Function to create an itinerary
def plan_itinerary(hotel, num_days, max_range_km):
    hotel_lat, hotel_lon = hotel["latitude"], hotel["longitude"]

    nearby_attractions = get_nearby_attractions(hotel_lat, hotel_lon, max_range_km).copy()
    nearby_restaurants = get_nearby_restaurants(hotel_lat, hotel_lon, max_range_km).copy()

    itinerary = []

    for day in range(1, num_days + 1):
        attractions_today = nearby_attractions.sample(n=min(len(nearby_attractions), random.randint(3, 4)), replace=False)
        nearby_attractions = nearby_attractions.drop(attractions_today.index)

        # Filter and sample restaurants for each meal within the loop
        breakfast = nearby_restaurants[nearby_restaurants["mealTypes"].str.contains("Breakfast", na=False)].sample(n=min(1, len(nearby_restaurants[nearby_restaurants["mealTypes"].str.contains("Breakfast", na=False)])), replace=False)
        lunch = nearby_restaurants[nearby_restaurants["mealTypes"].str.contains("Lunch", na=False)].sample(n=min(1,len(nearby_restaurants[nearby_restaurants["mealTypes"].str.contains("Lunch", na=False)])), replace=False)
        dinner = nearby_restaurants[nearby_restaurants["mealTypes"].str.contains("Dinner", na=False)].sample(n=min(1,len(nearby_restaurants[nearby_restaurants["mealTypes"].str.contains("Dinner", na=False)])), replace=False)

        # Drop the selected restaurants for the current day from the available options for the next day
        nearby_restaurants = nearby_restaurants.drop(breakfast.index.union(lunch.index).union(dinner.index), errors='ignore')

        itinerary.append({
            "Day": day,
            "Attractions": list(attractions_today["name"].values),
            "Breakfast": list(breakfast["name"].values),
            "Lunch": list(lunch["name"].values),
            "Dinner": list(dinner["name"].values),
        })

    return pd.DataFrame(itinerary)

# Sample user inputs
num_days = 3  # Number of days in trip
max_range_km = 5  # Max range from hotel in km

# Suggest hotels and select one
suggested_hotels = suggest_hotels()
selected_hotel = suggested_hotels.iloc[0]  # Assume user selects the top hotel

# Generate itinerary
itinerary_df = plan_itinerary(selected_hotel, num_days, max_range_km)

print(itinerary_df)
display(itinerary_df)


   Day                                        Attractions  \
0    1  [Seema Malakaya Temple, One Galle Face, IAMSRI...   
1    2  [Pettah, ZRI Adventures, Sri Lanka Airport Tra...   
2    3  [Sri Lanka Tour Driver By Praneeth, Doi Doi We...   

                        Breakfast                                 Lunch  \
0  [The Poolside Bar and Terrace]                              [Thalis]   
1                        [Plates]  [Chutneys At Cinnamon Grand Colombo]   
2   [Barista, World Trade Center]                   [Colombo Fort Cafe]   

                Dinner  
0          [Cloud Red]  
1  [Hotel de Pilawoos]  
2      [Emperor's Wok]  


,Day,Attractions,Breakfast,Lunch,Dinner
0,1,"[Seema Malakaya Temple, One Galle Face, IAMSRI...",[The Poolside Bar and Terrace],[Thalis],[Cloud Red]
1,2,"[Pettah, ZRI Adventures, Sri Lanka Airport Tra...",[Plates],[Chutneys At Cinnamon Grand Colombo],[Hotel de Pilawoos]
2,3,"[Sri Lanka Tour Driver By Praneeth, Doi Doi We...","[Barista, World Trade Center]",[Colombo Fort Cafe],[Emperor's Wok]
